In [ ]:
import polars as pl
import json

In [ ]:
with open('../params.json', 'r') as file :
    params = json.load(file)

DATASET, VERSION = params['dataset'], params['version']
DATA_FOLD = params['data_folder']

In [ ]:
DATA_FILE = f'{DATA_FOLD}/{VERSION}/2.clean_data/{DATASET}/temporal/all_features_with_delta.parquet'

In [ ]:
df = pl.read_parquet(DATA_FILE)

In [ ]:
df.head(5)

In [ ]:
df.columns[2:]

In [ ]:
df = df.with_columns(
    pl.when(pl.col("urine_output") == 0).then(pl.lit(None, dtype=pl.Float64)).otherwise(pl.col('urine_output')).alias('urine_output'),
    pl.when(pl.col("cgr") == 0).then(pl.lit(None, dtype=pl.Float64)).otherwise(pl.col('cgr')).alias('cgr'),
    pl.when(pl.col("pfc") == 0).then(pl.lit(None, dtype=pl.Float64)).otherwise(pl.col('pfc')).alias('pfc'),
    pl.when(pl.col("plq") == 0).then(pl.lit(None, dtype=pl.Float64)).otherwise(pl.col('plq')).alias('plq')
).filter(~pl.all_horizontal(pl.col(df.columns[2:]).is_null()))

In [ ]:
for col in df.columns[2:] :
    df_col = df.select('encounterId', 'delta_hour', col).filter(pl.col(col).is_not_null())
    print(f"{col} : {df_col['encounterId'].unique().shape[0]}/{df['encounterId'].unique().shape[0]}")